### TTim example of pumping test analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from ttim import *

Load data of two observation wells

In [ ]:
drawdown = np.loadtxt('data/oudekorendijk_h30.dat')
to1 = drawdown[:,0] / 60 / 24
ho1 = -drawdown[:,1]
ro1 = 30

drawdown = np.loadtxt('data/oudekorendijk_h90.dat')
to2 = drawdown[:,0] / 60 / 24
ho2 = -drawdown[:,1]
ro2 = 90

Pumping discharge

In [ ]:
Qo = 788

Create model

In [ ]:
ml = ModelMaq(kaq=60, z=(-18, -25), Saq=1e-4, tmin=1e-5, tmax=1)
w = Well(ml, xw=0, yw=0, rw=0.1, tsandQ=[(0, 788)], layers=0)
ml.solve()

Create calibration object, add parameters and first series. Fit the model. The chi-square value is the sum of squared residuals at the optimum.

In [ ]:
cal = Calibrate(ml)
cal.parameter(name='kaq0', initial=10)
cal.parameter(name='Saq0', initial=1e-4)
cal.series(name='obs1', x=ro1, y=0, layer=0, t=to1, h=ho1)
cal.fit()

In [ ]:
h = ml.head(ro1, 0, to1, 0)
plt.semilogx(to1, ho1, '.', label='observed')
plt.semilogx(to1, h[0], label='model')

### Specify $S$ rather than $S_s$

In [ ]:
cal = Calibrate(ml)
cal.parameter(name='kaq0', initial=10)
cal.parameter(name='S', par=ml.aq.Saq, layer=0, initial=1e-4)
cal.series(name='obs1', x=ro1, y=0, layer=0, t=to1, h=ho1)
cal.fit()

### Add wellbore storage

In [ ]:
ml = ModelMaq(kaq=60, z=(-18, -25), Saq=1e-4, tmin=1e-5, tmax=1)
w = Well(ml, xw=0, yw=0, rw=0.1, rc=0.2, tsandQ=[(0, 788)], layers=0)
ml.solve()

In [ ]:
cal = Calibrate(ml)
cal.parameter(name='kaq0', initial=10)
cal.parameter(name='Saq0', initial=1e-4)
cal.parameter(name='rc', par=w.rc, initial=0.2, pmin=0, pmax=0.5)
cal.series(name='obs1', x=ro1, y=0, layer=0, t=to1, h=ho1)
cal.fit()

In [ ]:
h = ml.head(ro1, 0, to1, 0)
plt.semilogx(to1, ho1, '.', label='observed')
plt.semilogx(to1, h[0], label='model')

### Two observation wells

In [ ]:
cal = Calibrate(ml)
cal.parameter(name='kaq0', initial=10)
cal.parameter(name='Saq0', initial=1e-4)
cal.parameter(name='rc', par=w.rc, initial=0.2, pmin=0, pmax=0.5)
cal.series(name='obs1', x=ro1, y=0, layer=0, t=to1, h=ho1)
cal.series(name='obs2', x=ro2, y=0, layer=0, t=to2, h=ho2)
cal.fit()

In [ ]:
h1 = ml.head(ro1, 0, to1, 0)
plt.semilogx(to1, ho1, '.', label='obs1')
plt.semilogx(to1, h1[0], label='model')
h2 = ml.head(ro2, 0, to2, 0)
plt.semilogx(to2, ho2, '.', label='obs2')
plt.semilogx(to2, h2[0], label='model')